In [155]:
import pandas as pd
import glob
import os

# Install pyarrow by conda prompt: 
# conda install -c conda-forge pyarrow 
# or by pip
#!pip install pyarrow

import pyarrow.parquet as pq
import pyarrow as pa

In [160]:
#Please define your path here: 
#Reminder: YOUR NOTEBOOK must be in the same place, where the data folder ist. Furthermore you need two sub folders 
# within the data folder: 1) input (place where the csv files are) 2) output (where your parquets will be)
path = r'C:\Users\Davis\Desktop\AAA\data\input'

In [161]:
#Set year
year = "2017"

In [166]:
# Data structure: Put folder "data" to the directory, where the 
def load2017data(path,year):

    #get number of files in directory: 
    dirListing = os.listdir(path)
    print("Amount of csv files: ",len(dirListing))

    #initialize divider - helper for the loopings and the naming of the parquet files
    csv_amount = len(dirListing)
    divider_counter = int(csv_amount/50)
    stayover = csv_amount-(divider_counter*50)
    print("Divider: ",50)
    print("Divider counter: ",divider_counter)
    print("Remaining files: ",stayover)

    #Define start and end for loops
    start = 1
    end = 51

    for j in range(1,divider_counter+1):
        #Range is in 50 steps! - Please limit your csv files to 1 mio rows!
        for i in range (start,end):
            all_files = glob.glob(path + "/trips.{}.csv".format(i))
            print(all_files)

            li = []

            for filename in all_files:
                df = pd.read_csv(filename, index_col=None, header=0)
                li.append(df)
        frame = pd.concat(li, axis=0, ignore_index=True)
        frame = frame[frame.iloc[:, 2].str.contains(year) | frame.iloc[:, 3].str.contains("2017")]
        frame = pa.Table.from_pandas(frame)
        pq.write_table(frame, 'data/output/trips_{}.parquet'.format(j))
        print("File saved as: trips_{}.parquet".format(j))
        #caching.clear_cache()

        if j < divider_counter:
            start = end
            end = end +50
            
        # Delete cache
        del frame
        del li
        del df 
        del all_files

    start = end 
    end = end+ stayover-1
    print()
    print("New start: ",start)
    print("New end: ",end)
    print()

    # Add all files, which are remaning outside of the counter
    for i in range(start,end):
        all_files = glob.glob(path + "/trips.{}.csv".format(i))
        print(all_files)

        li = []

        for filename in all_files:
            df = pd.read_csv(filename, index_col=None, header=0)
            li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    frame = frame[frame.iloc[:, 2].str.contains(year) | frame.iloc[:, 3].str.contains("2017")]
    frame = pa.Table.from_pandas(frame)
    pq.write_table(frame, 'data/output/trips_{}.parquet'.format(divider_counter+1))
    print("File saved as: trips_{}.parquet".format(divider_counter+1))
    
    # Delete cache
    del frame
    del li
    del df 
    del all_files
    print()

    print()
    print("Finished!")

In [167]:
#Run function
load2017data(path,year)

Amount of csv files:  85
Divider:  50
Divider counter:  1
Remaining files:  35
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.1.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.2.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.3.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.4.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.5.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.6.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.7.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.8.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.9.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.10.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.11.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.12.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.13.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.14.csv']
['C:\\Users\\Davis\\Desktop\\AAA\\data\\input/trips.15.csv']
['C:\\Users\\Da

# Reading csv files

In [145]:
#Read files
path = r'C:\Users\Davis\Desktop\AAA\data\output\*.parquet'
files = glob.glob(path)
files.sort()
df = pd.concat([pd.read_parquet(fp) for fp in files])

In [147]:
df.info(100)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1361 entries, 0 to 399
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Trip ID                     1361 non-null   object 
 1   Taxi ID                     1361 non-null   object 
 2   Trip Start Timestamp        1361 non-null   object 
 3   Trip End Timestamp          1361 non-null   object 
 4   Trip Seconds                1361 non-null   float64
 5   Trip Miles                  1361 non-null   float64
 6   Pickup Census Tract         863 non-null    float64
 7   Dropoff Census Tract        862 non-null    float64
 8   Pickup Community Area       1226 non-null   float64
 9   Dropoff Community Area      1192 non-null   float64
 10  Fare                        1361 non-null   float64
 11  Tips                        1361 non-null   float64
 12  Tolls                       1361 non-null   float64
 13  Extras                      1361 n

In [154]:
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,ffcb93a4f5cc1a9308067f22e2665757be6f4c29,f9f12d79733b1fa7934f8d9bd17ca1927f3c99ded1640b...,11/16/2019 01:15:00 AM,11/16/2019 01:15:00 AM,480.0,2.30,1.703108e+10,1.703124e+10,8.0,24.0,...,0.0,9.25,Cash,Medallion Leasin,41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),41.906026,-87.675312,POINT (-87.6753116216 41.906025969)
1,fed596275768c45a8b9c72d6ca99adf80fca7f69,435806033762c190275390c7a14f6d9f50a894562b1139...,11/29/2019 12:00:00 AM,11/29/2019 12:00:00 AM,600.0,4.60,NaN,NaN,41.0,33.0,...,0.0,14.25,Cash,Taxi Affiliation Services,41.794090,-87.592311,POINT (-87.592310855 41.794090253),41.857184,-87.620335,POINT (-87.6203346241 41.8571838585)
2,ffe09dca4b08bd2a9ecadf8634b163c2f856cb76,fc9af5a263f70826b274b29067232130b35f23b91479bb...,11/08/2019 03:45:00 PM,11/08/2019 04:00:00 PM,322.0,0.62,1.703184e+10,1.703132e+10,32.0,32.0,...,0.0,5.50,Cash,City Service,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918)
3,ff96942d2c5705211ca5363358cc1864c2f8ae9e,ac1f0defd00d6b7417e39256ebe1cad16b64955f9a64f2...,11/17/2019 05:15:00 AM,11/17/2019 05:15:00 AM,300.0,0.00,NaN,NaN,4.0,16.0,...,0.0,6.75,Cash,Taxi Affiliation Services,41.975171,-87.687516,POINT (-87.6875155152 41.9751709433),41.953582,-87.723452,POINT (-87.7234523905 41.9535821253)
4,fed981229d47fac008920f9402cd094dc711df9b,2b4bd3ac2b2acea1f6babf5043a743a308235faad57504...,11/13/2019 10:15:00 AM,11/13/2019 10:15:00 AM,180.0,0.30,1.703108e+10,1.703108e+10,8.0,8.0,...,0.0,4.50,Cash,Medallion Leasin,41.905858,-87.630865,POINT (-87.6308650266 41.9058577688),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134)
